# Common (Polyglot)

In [ ]:
#!import ../nbs/Testing.dib

In [ ]:
let nl = System.Environment.NewLine
let q = @""""

In [ ]:
let inline cons head tail = head :: tail

In [ ]:
module String =
    let inline contains (value : string) (input : string) =
        input.Contains value

    let inline endsWith (value : string) (input : string) =
        input.EndsWith value

    let inline padLeft totalWidth paddingChar (input : string) =
        input.PadLeft (totalWidth, paddingChar)

    let inline replace (oldValue : string) (newValue : string) (input : string) =
        input.Replace (oldValue, newValue)

    let inline split separator (input : string) =
        input.Split separator

    let inline splitString (separator : string array) (input : string) =
        input.Split (separator, System.StringSplitOptions.None)

    let inline startsWith (value : string) (input : string) =
        input.StartsWith value

    let inline substring startIndex length (input : string) =
        input.Substring (startIndex, length)

    let inline trim (input : string) =
        input.Trim ()
    
    let inline trimEnd (trimChars : char array) (input : string) =
        input.TrimEnd trimChars

    let inline trimStart (trimChars : char array) (input : string) =
        input.TrimStart trimChars

In [ ]:
type TicksGuid = System.Guid
type DateTimeGuid = System.Guid

In [ ]:
//// test

let testGuid = Guid "FEDCBA98-7654-3210-FEDC-BA9876543210"

## dateTimeGuidFromDateTime

In [ ]:
let inline dateTimeGuidFromDateTime (guid: System.Guid) (dateTime: System.DateTime) =
    let guid = guid |> string
    let prefix = dateTime.ToString "yyyyMMdd-HHmm-ssff-ffff-f"
    DateTimeGuid $"{prefix}{guid.[prefix.Length..]}"

In [ ]:
//// test

dateTimeGuidFromDateTime testGuid DateTime.MinValue
|> _equal (DateTimeGuid "00010101-0000-0000-0000-0a9876543210")

00010101-0000-0000-0000-0a9876543210


In [ ]:
//// test

dateTimeGuidFromDateTime testGuid DateTime.MaxValue
|> _equal (DateTimeGuid $"99991231-2359-5999-9999-9{(testGuid |> string).[^10..]}")

99991231-2359-5999-9999-9a9876543210


In [ ]:
//// test

dateTimeGuidFromDateTime testGuid DateTime.UnixEpoch
|> _equal (DateTimeGuid $"19700101-0000-0000-0000-0{(testGuid |> string).[^10..]}")

19700101-0000-0000-0000-0a9876543210


## dateTimeFromGuid

In [ ]:
let inline dateTimeFromGuid (dateTimeGuid: DateTimeGuid) =
    let dateTimeGuid = dateTimeGuid |> string
    System.DateTime.ParseExact (dateTimeGuid.[..24] |> String.replace "-" "", "yyyyMMddHHmmssfffffff", null)

In [ ]:
//// test

dateTimeFromGuid (DateTimeGuid "00010101-0000-0000-0000-0a9876543210")
|> _equal DateTime.MinValue

0001-01-01 00:00:00Z


In [ ]:
//// test

dateTimeFromGuid (DateTimeGuid $"99991231-2359-5999-9999-9{(testGuid |> string).[^10..]}")
|> _equal DateTime.MaxValue

9999-12-31 23:59:59Z


In [ ]:
//// test

dateTimeFromGuid (DateTimeGuid $"19700101-0000-0000-0000-0{(testGuid |> string).[^10..]}")
|> _equal DateTime.UnixEpoch

1970-01-01 00:00:00Z


## ticksGuidFromTicks

In [ ]:
let inline ticksGuidFromTicks (guid: System.Guid) (ticks: int64) =
    let guid = guid |> string
    let ticks = ticks |> string |> String.padLeft 18 '0'
    TicksGuid $"{ticks.[0..7]}-{ticks.[8..11]}-{ticks.[12..15]}-{ticks.[16..17]}{guid.[21..]}"

In [ ]:
//// test

ticksGuidFromTicks testGuid 0L
|> _equal (TicksGuid "00000000-0000-0000-00dc-ba9876543210")

00000000-0000-0000-00dc-ba9876543210


In [ ]:
//// test

ticksGuidFromTicks testGuid 999999999999999999L
|> _equal (TicksGuid $"99999999-9999-9999-99dc-b{(testGuid |> string).[^10..]}")

99999999-9999-9999-99dc-ba9876543210


## ticksFromGuid

In [ ]:
let inline ticksFromGuid (ticksGuid: DateTimeGuid) =
    let ticks = ticksGuid |> string
    int64 $"{ticks.[0..7]}{ticks.[9..12]}{ticks.[14..17]}{ticks.[19..20]}"

In [ ]:
//// test

ticksFromGuid (TicksGuid "00000000-0000-0000-00dc-ba9876543210")
|> _equal 0L

0


In [ ]:
//// test

ticksFromGuid (TicksGuid $"99999999-9999-9999-99{(testGuid |> string).[^14..]}")
|> _equal 999999999999999999L

999999999999999999


## newGuidFromDateTime

In [ ]:
let inline newGuidFromDateTime (dateTime: System.DateTime) =
    let guid = System.Guid.NewGuid ()
    dateTimeGuidFromDateTime guid dateTime

In [ ]:
//// test

newGuidFromDateTime System.DateTime.UtcNow
|> dateTimeFromGuid
|> fun dateTime -> (dateTime - System.DateTime.UtcNow).TotalSeconds |> int
|> _equal 0

0


## newGuidFromTicks

In [ ]:
let inline newGuidFromTicks (ticks: int64) =
    let guid = System.Guid.NewGuid ()
    ticksGuidFromTicks guid ticks

In [ ]:
//// test

newGuidFromTicks System.DateTime.UtcNow.Ticks
|> ticksFromGuid
|> fun ticks -> (ticks - System.DateTime.UtcNow.Ticks) / 100000L
|> _equal 0L

0


## memoize

In [ ]:
let inline memoize fn =
    let result = lazy fn ()
    fun () -> result.Value

In [ ]:
//// test

let mutable count = 0
let add =
    fun () -> count <- count + 1
    |> memoize

add ()
add ()
add ()

count
|> _equal 1

1


## trace

In [ ]:
type TraceLevel =
    | Verbose
    | Debug
    | Info
    | Warn
    | Error

let inline getLocals () = ""

let mutable traceEnabled = true
let mutable traceCount = 0
let mutable traceLevel = Verbose

let private replStart =
    fun () ->
        if System.Reflection.Assembly.GetEntryAssembly().GetName().Name = "dotnet-repl"
        then Some System.DateTime.Now
        else None
    |> memoize

let trace level fn getLocals =
    if traceEnabled && level >= traceLevel then
        traceCount <- traceCount + 1
        let time =
            match replStart () with
            | Some dateTime ->
                let t = System.DateTime.Now - dateTime
                System.DateTime (1, 1, 1, t.Hours, t.Minutes, t.Seconds, t.Milliseconds, t.Microseconds)
            | None -> System.DateTime.Now
            |> fun dateTime -> dateTime.ToString "HH:mm:ss"
        $"{time} #{traceCount} [{level}] {fn ()} / {getLocals ()}"
        |> String.trimEnd [| ' '; '/' |]
        |> System.Console.WriteLine

let inline withTrace enabled fn =
    let oldTraceEnabled = traceEnabled
    try
        traceEnabled <- enabled
        fn ()
    finally
        traceEnabled <- oldTraceEnabled

let inline withTraceEnabled fn =
    withTrace true fn

let inline withTraceDisabled fn =
    withTrace false fn

let inline withTraceLevel level fn =
    let oldTraceLevel = traceLevel
    try
        traceLevel <- level
        fn ()
    finally
        traceLevel <- oldTraceLevel

In [ ]:
//// test

trace Debug (fun () -> "test") getLocals

00:00:00 #1 [Debug] test


## newDisposable

In [ ]:
let inline newDisposable fn =
    { new System.IDisposable with
        member _.Dispose () = fn ()
    }

In [ ]:
//// test

let mutable newDisposableTest = 0
newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
|> fun x -> x.Dispose ()
newDisposableTest |> _equal 1

1


In [ ]:
//// test

let mutable newDisposableTest = 0
fun () -> task {
    use x = newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
    ()
}
|> fun x -> x () |> Async.AwaitTask |> Async.RunSynchronously
newDisposableTest |> _equal 1

1


In [ ]:
//// test

let mutable newDisposableTest = 0
async {
    use x = newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
    ()
}
|> Async.RunSynchronously
newDisposableTest |> _equal 1

1


In [ ]:
//// test

let mutable newDisposableTest = 0
async {
    let x = newDisposable (fun () -> newDisposableTest <- newDisposableTest + 1)
    ()
}
|> Async.RunSynchronously
newDisposableTest |> _equal 0

0


## printException

In [ ]:
let inline printException (ex : System.Exception) =
    $"{ex.GetType ()}: {ex.Message}"

In [ ]:
//// test

fun () -> failwith "test"
|> _throwsC (fun ex _ ->
    printException ex
    |> _equal "System.Exception: test"
)

FSI_0041+it@3-3
System.Exception: test


## retryFn

In [ ]:
let inline retryFn retries fn =
    let rec loop retry =
        try
            if retry < retries
            then fn () |> Some
            else None
        with ex ->
            let getLocals () = $"retry: {retry} / ex: {ex |> printException} / {getLocals ()}"
            trace Warn (fun () -> "retryFn") getLocals
            System.Threading.Thread.Sleep 1
            loop (retry + 1)
    loop 0

In [ ]:
//// test

let mutable retryFnTest = 0
fun () ->
    retryFnTest <- retryFnTest + 1
    retryFnTest
|> retryFn 3
|> _equal (Some 1)

FSharpOption<Int32>
      Value: 1


In [ ]:
//// test

fun () -> failwith "test"
|> retryFn 3
|> _equal None

00:00:00 #2 [Warn] retryFn / retry: 0 / ex: System.Exception: test
00:00:00 #3 [Warn] retryFn / retry: 1 / ex: System.Exception: test
00:00:00 #4 [Warn] retryFn / retry: 2 / ex: System.Exception: test
<null>


In [ ]:
//// test

let mutable retryFnTest = 0
fun () ->
    if retryFnTest >= 2
    then retryFnTest
    else
        retryFnTest <- retryFnTest + 1
        failwith "test"
|> retryFn 3
|> _equal (Some 2)

00:00:00 #5 [Warn] retryFn / retry: 0 / ex: System.Exception: test
00:00:00 #6 [Warn] retryFn / retry: 1 / ex: System.Exception: test
FSharpOption<Int32>
      Value: 2


## getUnionCaseName

In [ ]:
let getUnionCaseName (x: 'T) =
    match FSharp.Reflection.FSharpValue.GetUnionFields(x, typeof<'T>) with
    | case, _ -> case.Name

In [ ]:
//// test

TraceLevel.Error
|> getUnionCaseName
|> _equal (nameof TraceLevel.Error)

Error
